In [1]:
from importlib import reload
def reload_all():
    import tsm; reload(tsm)
    import _keys; reload(_keys)
    import blizzard; reload(blizzard)
    import config; reload(config)
    import kvstore; reload(kvstore)
    import snapshot; reload(snapshot)
    import elemental_arbitrage; reload(elemental_arbitrage)
    import bliz_tsm_join; reload(bliz_tsm_join)
    import crafting; reload(crafting)
    import kvstore; reload(kvstore)
    import crafting; reload(crafting)

In [2]:
from snapshot import SnapshotProcessor
from tsm import auction_house_snapshot
from config import tsm_region_id
from config import tsm_realm_id
from config import tsm_ah_id
from config import tsm_cache_dir

def tsm_ah_snapper():
    return auction_house_snapshot(tsm_region_id, tsm_realm_id, tsm_ah_id)

tsm_ah_snap = SnapshotProcessor(tsm_ah_snapper, cache_dir=tsm_cache_dir)
tsm_ah = tsm_ah_snap.get(max_age_seconds=3000)

In [3]:
from snapshot import SnapshotProcessor
from blizzard import auction_data
from blizzard import auction_summary
from config import blizzard_realm_id
from config import blizzard_ah_id
from config import blizzard_cache_dir

def bliz_ah_snapper():
    return auction_data(blizzard_realm_id, blizzard_ah_id)

bliz_ah_snap = SnapshotProcessor(bliz_ah_snapper, cache_dir=blizzard_cache_dir)
bliz_ah = bliz_ah_snap.get(max_age_seconds=3000)

In [4]:
from kvstore import InefficientKVStore
from blizzard import ItemLookup
from config import blizzard_item_cache
from config import blizzard_item_reverse_cache

items = ItemLookup(
    InefficientKVStore(blizzard_item_cache),
    InefficientKVStore(blizzard_item_reverse_cache),
)

In [5]:
from bliz_tsm_join import item_info_getter
item_info = item_info_getter(items, bliz_ah_snap, tsm_ah_snap, max_age_seconds=3000)
item_info_current = item_info_getter(items, bliz_ah_snap, tsm_ah_snap, max_age_seconds=0)

In [6]:
reload_all()

from crafting import Recipes

r = Recipes(items)
i = r.ingredients


recipe_listing = """
2 flask of the frost wyrm
5 icethorn + 5 lichbloom + frost lotus + enchanted vial

2 flask of endless rage
7 lichbloom + 3 goldclover + frost lotus + enchanted vial

2 flask of stoneblood
7 lichbloom + 3 crystallized life + frost lotus + enchanted vial

firecracker salmon
glacial salmon + northern spices

elixir of greater agility
sungrass + goldthorn + crystal vial

titanium bar
8 saronite bar

titanium bar
2 titanium ore

titansteel bar
3 titanium bar + eternal fire + eternal earth + eternal shadow

indestructible potion
2 icethorn + imbued vial

saronite bar
2 saronite ore

potion of speed
2 adder's tongue + pygmy oil + imbued vial

potion of wild magic
2 lichbloom + pygmy oil + imbued vial
"""

reversible_recipes = """
eternal shadow
10 crystallized shadow

eternal water
10 crystallized water

greater planar essence
3 lesser planar essence

greater cosmic essence
3 lesser cosmic essence
"""

r_out = None
for line in recipe_listing.splitlines():
    if line.strip():
        if r_out:
            r.recipe_from_strings(r_out, line.strip())
            r_out = None
        else:
            r_out = line.strip()

            
r_out = None
for line in reversible_recipes.splitlines():
    if line.strip():
        if r_out:
            r.recipe_from_strings(r_out, line.strip())
            r.recipe_from_strings(line.strip(), r_out)
            r_out = None
        else:
            r_out = line.strip()

In [7]:
r.lookup(i("potion of speed"))

[([potion of speed], [pygmy oil] + [imbued vial] + 2*[adder's tongue])]

In [8]:
item_info("adder's tongue")

{'id': 36903,
 'name': "adder's tongue",
 'num_auctions': 58,
 'quantity': 947,
 'weight_sell': 19.601181098204858,
 'avg_sell': 16.440156896551724,
 'max': 23.398,
 'p80': 19.888,
 'p50': 19.884,
 'p20': 11.0066,
 'wp80': 19.888,
 'wp50': 19.888,
 'wp20': 19.884,
 'min': 1.0005,
 'realm_market_value': 1.1439,
 'realm_historical': 1.1516,
 'region_historical': 0.9732,
 'region_avg_sale_price': 0.9822,
 'sale_pct': 83.3,
 'sold_per_day': 52079,
 'headroom': 61572,
 'market_skew_pct': -0.67,
 'auction_skew_pct': -99.99}

In [9]:
item_info("pygmy oil")

{'id': 40195,
 'name': 'pygmy oil',
 'num_auctions': 73,
 'quantity': 480,
 'weight_sell': 14.444267083333333,
 'avg_sell': 7.1811849315068494,
 'max': 21.976,
 'p80': 8.788,
 'p50': 5.4945,
 'p20': 1.0873,
 'wp80': 21.976,
 'wp50': 21.7259,
 'wp20': 5.4945,
 'min': 1.0862,
 'realm_market_value': 1.4644,
 'realm_historical': 2.0042,
 'region_historical': 1.5555,
 'region_avg_sale_price': 1.6382,
 'sale_pct': 68.8,
 'sold_per_day': 2927,
 'headroom': 3774,
 'market_skew_pct': -26.93,
 'auction_skew_pct': -99.99}

In [10]:
item_info("potion of speed")

{'id': 40211,
 'name': 'potion of speed',
 'num_auctions': 169,
 'quantity': 1055,
 'weight_sell': 35.709345971563984,
 'avg_sell': 24.89157396449704,
 'max': 79.998,
 'p80': 20.0,
 'p50': 20.0,
 'p20': 20.0,
 'wp80': 78.478,
 'wp50': 20.0,
 'wp20': 20.0,
 'min': 19.619,
 'realm_market_value': 4.2317,
 'realm_historical': 6.4361,
 'region_historical': 4.789,
 'region_avg_sale_price': 4.1759,
 'sale_pct': 71.2,
 'sold_per_day': 7305,
 'headroom': 9204,
 'market_skew_pct': -34.25,
 'auction_skew_pct': -99.95}

In [11]:
from crafting import crafting_price
from crafting import gold

def approaches(item_name):
    market_craft = crafting_price(
        item_info,
        r.lookup(i(item_name))[0],
        in_price_method=gold("realm_market_value"),
        out_price_method=gold("realm_market_value"),
    )
    flip_craft = crafting_price(
        item_info,
        r.lookup(i(item_name))[0],
        in_price_method=gold("min"),
        out_price_method=gold("min"),
    )
    hold_craft = crafting_price(
        item_info,
        r.lookup(i(item_name))[0],
        in_price_method=gold("min"),
        out_price_method=gold("realm_market_value"),
    )
    return {"market": market_craft, "flip": flip_craft, "hold": hold_craft}

In [12]:
item_info("titansteel bar")

{'id': 37663,
 'name': 'titansteel bar',
 'num_auctions': 17,
 'quantity': 17,
 'weight_sell': 158.84235294117647,
 'avg_sell': 158.84235294117647,
 'max': 175.2563,
 'p80': 160.0,
 'p50': 159.9699,
 'p20': 149.9998,
 'wp80': 160.0,
 'wp50': 159.9699,
 'wp20': 149.9998,
 'min': 149.9996,
 'realm_market_value': 139.2969,
 'realm_historical': 145.5547,
 'region_historical': 119.2709,
 'region_avg_sale_price': 114.7697,
 'sale_pct': 80.5,
 'sold_per_day': 447,
 'headroom': 538,
 'market_skew_pct': -4.3,
 'auction_skew_pct': -99.99}

In [13]:
item_info("glyph of unburdened rebirth")

{'id': 43331,
 'name': 'glyph of unburdened rebirth',
 'num_auctions': 4,
 'quantity': 4,
 'weight_sell': 42.754125,
 'avg_sell': 42.754125,
 'max': 42.7543,
 'p80': 42.7543,
 'p50': 42.7541,
 'p20': 42.754,
 'wp80': 42.7543,
 'wp50': 42.7541,
 'wp20': 42.754,
 'min': 42.754,
 'realm_market_value': 39.0041,
 'realm_historical': 34.9882,
 'region_historical': 9.0844,
 'region_avg_sale_price': 6.5924,
 'sale_pct': 33.3,
 'sold_per_day': 57,
 'headroom': 167,
 'market_skew_pct': 11.48,
 'auction_skew_pct': -99.99}

In [14]:
item_info("glyph of typhoon")

{'id': 44922,
 'name': 'glyph of typhoon',
 'num_auctions': 2,
 'quantity': 2,
 'weight_sell': 45.9999,
 'avg_sell': 45.9999,
 'max': 45.9999,
 'p80': 45.9999,
 'p50': 45.9999,
 'p20': 45.9999,
 'wp80': 45.9999,
 'wp50': 45.9999,
 'wp20': 45.9999,
 'min': 45.9999,
 'realm_market_value': 7.0861,
 'realm_historical': 13.6495,
 'region_historical': 7.2497,
 'region_avg_sale_price': 5.7083,
 'sale_pct': 18.9,
 'sold_per_day': 26,
 'headroom': 135,
 'market_skew_pct': -48.09,
 'auction_skew_pct': -99.94}

In [15]:
from elemental_arbitrage import print_elemental_arbitrage_report

print_elemental_arbitrage_report(tsm_ah, items)

DEFINITELY   SB eternal-life (unit profit: 10223,headroom: 7166)
DEFINITELY   BS eternal-earth (unit profit: 55405,headroom: 4046)
DEFINITELY   BS eternal-air (unit profit: 10414,headroom: 2685)
DEFINITELY   BS eternal-fire (unit profit: 43063,headroom: 2005)
DEFINITELY   BS eternal-water (unit profit: 36239,headroom: 3836)
DEFINITELY   BS eternal-shadow (unit profit: 30516,headroom: 4650)
SKIP        primal-life
SKIP        primal-earth
SKIP        primal-air
SKIP        primal-fire
SKIP        primal-water
SKIP        primal-shadow
DEFINITELY   BS essence-eternal (unit profit: 78672,headroom: 631)
DEFINITELY   BS essence-cosmic (unit profit: 448884,headroom: 673)
SKIP        essence-nether
YES          BS essence-planar (unit profit: 26710,headroom: 732)
YES          BS essence-astral (unit profit: 51772,headroom: 447)
SKIP        essence-magic
DEFINITELY   BS essence-mystic (unit profit: 48035,headroom: 866)


In [16]:
item_info("crystallized water")

{'id': 37705,
 'name': 'crystallized water',
 'num_auctions': 31,
 'quantity': 179,
 'weight_sell': 6.477915083798883,
 'avg_sell': 4.97896129032258,
 'max': 9.936,
 'p80': 9.935,
 'p50': 3.75,
 'p20': 3.75,
 'wp80': 9.935,
 'wp50': 3.75,
 'wp20': 3.75,
 'min': 0.5494,
 'realm_market_value': 0.518,
 'realm_historical': 0.7524,
 'region_historical': 0.5107,
 'region_avg_sale_price': 0.4659,
 'sale_pct': 73.1,
 'sold_per_day': 2822,
 'headroom': 3681,
 'market_skew_pct': -31.15,
 'auction_skew_pct': -99.99}

In [17]:
found_sholazar = {"adder's": 3*20+15, "goldclover": 2*20+9, "frost lotus": 3, "deadnettle": 20+3, "tiger lily": 2*20+2, "crystallized life": 10*5+3}
sholazar_hpm = {k: v/50 for (k, v) in found_sholazar.items()}
sholazar_hpm

{"adder's": 1.5,
 'goldclover': 0.98,
 'frost lotus': 0.06,
 'deadnettle': 0.46,
 'tiger lily': 0.84,
 'crystallized life': 1.06}

In [18]:
found_wintergrasp = {'lichbloom': 26,
 'tiger_lily': 19,
 'icethorn': 27,
 'goldclover': 18,
 'frost_lotus': 3,
 "talandra's rose": 6,
 'deadnettle': 1}
wintergrasp_hpm = {k: v/30 for (k, v) in found_wintergrasp.items()}
wintergrasp_hpm

{'lichbloom': 0.8666666666666667,
 'tiger_lily': 0.6333333333333333,
 'icethorn': 0.9,
 'goldclover': 0.6,
 'frost_lotus': 0.1,
 "talandra's rose": 0.2,
 'deadnettle': 0.03333333333333333}

In [19]:
import pandas as pd
df = pd.read_csv("tsm-dec-13.csv")

In [20]:
from cytoolz import groupby
entries = list(df.transpose().to_dict().values())
grouped = groupby(lambda entry: entry["itemString"], entries)

In [21]:
names = {entry["itemString"]: entry["itemName"] for entry in entries}
nets = {k: sum(vv["net"] for vv in v) for (k, v) in grouped.items()}
ah_events = {k: [vv["event"] for vv in v if vv["source"] != "Vendor"] for (k, v) in grouped.items()}
sale_rate = {
    k: v.count("sale") / (v.count("sale") + v.count("expired") + v.count("cancelled"))
    for (k, v) in ah_events.items()
    if v.count("sale") > 1
}
agg = {k: {"name": names[k], "net": nets[k], "rate": sale_rate[k], "num": len(ah_events[k])} for k in sale_rate}
sorted(
    agg.values(),
    key=lambda entry: (entry["rate"], entry["net"]),
    reverse=True,
)

[{'name': 'Titanium Bar', 'net': 2049353, 'rate': 1.0, 'num': 9},
 {'name': 'Indestructible Potion', 'net': 1108163, 'rate': 1.0, 'num': 3},
 {'name': 'Swiftthistle', 'net': 708342, 'rate': 1.0, 'num': 5},
 {'name': 'Eternal Shadow', 'net': 630845, 'rate': 1.0, 'num': 2},
 {'name': 'Oily Blackmouth', 'net': 174990, 'rate': 1.0, 'num': 3},
 {'name': 'Elixir of Mighty Thoughts', 'net': 161775, 'rate': 1.0, 'num': 2},
 {'name': 'Frostweave Cloth', 'net': 120855, 'rate': 1.0, 'num': 3},
 {'name': 'Wildvine', 'net': 72204, 'rate': 1.0, 'num': 5},
 {'name': 'Felcloth', 'net': 69734, 'rate': 1.0, 'num': 7},
 {'name': 'Lesser Healing Potion', 'net': 1075, 'rate': 1.0, 'num': 3},
 {'name': 'Flask of Endless Rage', 'net': 6943980, 'rate': 0.875, 'num': 9},
 {'name': 'Runic Mana Potion', 'net': 5488, 'rate': 0.875, 'num': 10},
 {'name': 'Greater Planar Essence',
  'net': 956155,
  'rate': 0.8333333333333334,
  'num': 7},
 {'name': "Adder's Tongue",
  'net': 731355,
  'rate': 0.8333333333333334,
 

In [22]:
item_info("adamantite ore")

{'id': 23425,
 'name': 'adamantite ore',
 'num_auctions': 27,
 'quantity': 380,
 'weight_sell': 32.47681578947368,
 'avg_sell': 25.633351851851852,
 'max': 45.0,
 'p80': 34.998,
 'p50': 31.9999,
 'p20': 11.2485,
 'wp80': 35.0,
 'wp50': 34.998,
 'wp20': 31.6,
 'min': 1.5799,
 'realm_market_value': 2.4061,
 'realm_historical': 2.1965,
 'region_historical': 1.3753,
 'region_avg_sale_price': 1.4342,
 'sale_pct': 69.7,
 'sold_per_day': 5381,
 'headroom': 7340,
 'market_skew_pct': 9.54,
 'auction_skew_pct': -99.99}

In [23]:
item_info("infinite dust")

{'id': 34054,
 'name': 'infinite dust',
 'num_auctions': 126,
 'quantity': 1772,
 'weight_sell': 20.592333239277654,
 'avg_sell': 17.442665873015873,
 'max': 28.984,
 'p80': 25.0,
 'p50': 13.2168,
 'p20': 11.011,
 'wp80': 25.97,
 'wp50': 22.4802,
 'wp20': 12.495,
 'min': 1.2492,
 'realm_market_value': 1.6972,
 'realm_historical': 2.2993,
 'region_historical': 1.4893,
 'region_avg_sale_price': 1.2115,
 'sale_pct': 71.6,
 'sold_per_day': 21122,
 'headroom': 27728,
 'market_skew_pct': -26.19,
 'auction_skew_pct': -99.99}

In [24]:
item_info("swiftthistle")

{'id': 2452,
 'name': 'swiftthistle',
 'num_auctions': 23,
 'quantity': 234,
 'weight_sell': 19.378181623931624,
 'avg_sell': 11.889091304347826,
 'max': 40.0,
 'p80': 23.0,
 'p50': 9.99,
 'p20': 0.5,
 'wp80': 29.0,
 'wp50': 15.997,
 'wp20': 9.99,
 'min': 0.4996,
 'realm_market_value': 2.337,
 'realm_historical': 1.0999,
 'region_historical': 1.5159,
 'region_avg_sale_price': 2.3334,
 'sale_pct': 68.2,
 'sold_per_day': 928,
 'headroom': 1126,
 'market_skew_pct': 112.47,
 'auction_skew_pct': -100.0}

In [25]:
item_info("eternal shadow")

{'id': 35627,
 'name': 'eternal shadow',
 'num_auctions': 10,
 'quantity': 200,
 'weight_sell': 60.0,
 'avg_sell': 60.0,
 'max': 60.0,
 'p80': 60.0,
 'p50': 60.0,
 'p20': 60.0,
 'wp80': 60.0,
 'wp50': 60.0,
 'wp20': 60.0,
 'min': 60.0,
 'realm_market_value': 3.3984,
 'realm_historical': 3.2774,
 'region_historical': 2.6258,
 'region_avg_sale_price': 1.8142,
 'sale_pct': 66.4,
 'sold_per_day': 3698,
 'headroom': 5369,
 'market_skew_pct': 3.69,
 'auction_skew_pct': -99.82}

In [26]:
item_info("frostweave cloth")

{'id': 33470,
 'name': 'frostweave cloth',
 'num_auctions': 421,
 'quantity': 7831,
 'weight_sell': 10.720491903971396,
 'avg_sell': 10.432871733966746,
 'max': 25.0,
 'p80': 9.796,
 'p50': 9.788,
 'p20': 9.686,
 'wp80': 9.794,
 'wp50': 9.788,
 'wp20': 9.686,
 'min': 0.9999,
 'realm_market_value': 0.472,
 'realm_historical': 0.5044,
 'region_historical': 0.4273,
 'region_avg_sale_price': 0.4278,
 'sale_pct': 87.3,
 'sold_per_day': 29883,
 'headroom': 26399,
 'market_skew_pct': -6.42,
 'auction_skew_pct': -99.98}

In [27]:
item_info("mageweave cloth")

{'id': 4338,
 'name': 'mageweave cloth',
 'num_auctions': 18,
 'quantity': 325,
 'weight_sell': 47.69575230769231,
 'avg_sell': 45.13482777777777,
 'max': 50.0,
 'p80': 49.998,
 'p50': 49.994,
 'p20': 42.4983,
 'wp80': 49.998,
 'wp50': 49.994,
 'wp20': 49.994,
 'min': 9.9992,
 'realm_market_value': 0.5113,
 'realm_historical': 0.612,
 'region_historical': 0.5101,
 'region_avg_sale_price': 0.4432,
 'sale_pct': 66.8,
 'sold_per_day': 8747,
 'headroom': 12769,
 'market_skew_pct': -16.45,
 'auction_skew_pct': -99.8}

In [28]:
tsm_ah[4338]

{'auctionHouseId': 351,
 'itemId': 4338,
 'petSpeciesId': None,
 'minBuyout': 2000,
 'quantity': 21,
 'marketValue': 5113,
 'historical': 6120,
 'numAuctions': 2,
 'region': {'regionId': 13,
  'itemId': 4338,
  'petSpeciesId': None,
  'quantity': 1309.5,
  'marketValue': 5001,
  'avgSalePrice': 4432,
  'salePct': 66.8,
  'soldPerDay': 8747,
  'historical': 5101}}

In [29]:
item_info("wildvine")

KeyError: 8153

In [30]:
import pickle
from cytoolz import get as pick_from_dict
from cytoolz import sliding_window
import glob
import itertools
from blizzard import auction_summary
import re
import numpy as np
import datetime
np.set_printoptions(suppress=True)

_names = """
flask of the frost wyrm
flask of endless rage
flask of stoneblood
titanium bar
indestructible potion
eternal shadow
oily blackmouth
elixir of mighty thoughts
frostweave cloth
wildvine
felcloth
lesser healing potion
runic mana potion
greater planar essence
netherweave cloth
greater healing potion
crystallized water
purple lotus
bruiseweed
arcane dust
tiger lily
crystallized fire
major healing potion
potion of speed
potion of wild magic
lichbloom
pygmy oil
adder's tongue
imbued vial
icethorn
frost lotus
enchanted vial
goldclover
crystallized life
eternal life
deadnettle
elixir of accuracy
lesser planar essence
eternal fire
eternal water
""".strip().splitlines()
desired_names = _names
desired_ids = [items.get_id(name) for name in desired_names if not name.startswith("#")]


date_re = re.compile(r"2022-..-..T..-..-..")
def _timestamp_from_name(name):
    cap = date_re.search(name).group(0)
    return datetime.datetime.strptime(cap, "%Y-%m-%dT%H-%M-%S").timestamp()


by_item = {}
timeseries = []

for snapshot in sorted(glob.glob("bliz-ah/*")):
    with open(snapshot, "rb") as f:
        entries = pickle.load(f).values()
        timeseries.extend(
            [
                {"timestamp": _timestamp_from_name(snapshot), "item_name": items.get_name(item_id), **auction_summary(item_data)}
                for (item_id, item_data) in zip(desired_ids, entries)
            ]
        )
        for (item_id, item_data) in zip(desired_ids, entries):
            name = items.get_name(item_id)
            by_item[name] = by_item.get(name, []) + [{"timestamp": date_re.search(snapshot).group(0), "item_name": items.get_name(item_id), **auction_summary(item_data)}]

def timeseries_of(key, default=np.nan):
    def _timeseries_of(item_name):
        return np.array([v.get(key, default) for v in by_item[item_name]])
    return _timeseries_of

def omit_outliers(series, sigma=2, window=5):
    mavg = [np.average(w) for w in sliding_window(window, series)]
    std = np.std(mavg)
    return np.array([x if np.abs(x) < sigma*std else np.nan for x in mavg])

def avg_finite(arr):
    return np.average(arr[np.isfinite(arr)])

In [31]:
import plotly.express as px
for yvalue in ["num", "min", "p50", "d_num", "d_min", "d_p50"]:
    df = pd.DataFrame(timeseries)
    df = df.sort_values(by="timestamp")

    if yvalue.startswith("d_"):
        dfg = df.pivot(index="timestamp", columns="item_name", values=yvalue[2:])
        dfg = dfg.diff().div(dfg.index.to_series().diff(), axis=0)
    else:
        dfg = df.pivot(index="timestamp", columns="item_name", values=yvalue)

    fig = px.scatter(dfg, title=yvalue, log_y=False)
    fig.write_html(f"{yvalue}.html")

In [32]:
def min_series(x):
    return omit_outliers(timeseries_of("avg_sell")(x), sigma=0.5, window=10)

wild_magic_pot = avg_finite(min_series("potion of wild magic") - (2*min_series("lichbloom") + min_series("pygmy oil") + np.array(1.8)))
speed_pot = avg_finite(min_series("potion of speed") - (2*min_series("adder's tongue") + min_series("pygmy oil") + np.array(1.8)))
wyrm = avg_finite(2*min_series("flask of the frost wyrm") - (5*min_series("icethorn") + 5*min_series("lichbloom") + min_series("frost lotus") + np.array(5)))
rage = avg_finite(2*min_series("flask of endless rage") - (5*min_series("goldclover") + 7*min_series("lichbloom") + min_series("frost lotus") + np.array(5)))
indestructible = avg_finite(2*min_series("indestructible potion") - (2*min_series("icethorn") + np.array(1.8)))

C:\Users\Med\anaconda3\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning:

Mean of empty slice.

C:\Users\Med\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning:

invalid value encountered in double_scalars



In [33]:
wild_magic_pot, speed_pot, wyrm, rage, indestructible

(-24.551711954744242, -32.537304697483975, nan, nan, nan)

In [34]:
item_info("earthsiege diamond")

{'id': 41334,
 'name': 'earthsiege diamond',
 'num_auctions': 7,
 'quantity': 7,
 'weight_sell': 39.39837142857142,
 'avg_sell': 39.39837142857142,
 'max': 39.3986,
 'p80': 39.3986,
 'p50': 39.3985,
 'p20': 39.3981,
 'wp80': 39.3986,
 'wp50': 39.3985,
 'wp20': 39.3981,
 'min': 39.3981,
 'realm_market_value': 37.4444,
 'realm_historical': 32.8327,
 'region_historical': 29.1809,
 'region_avg_sale_price': 22.5086,
 'sale_pct': 48.8,
 'sold_per_day': 222,
 'headroom': 447,
 'market_skew_pct': 14.05,
 'auction_skew_pct': -99.99}

In [35]:
item_info("flask of stoneblood")

{'id': 46379,
 'name': 'flask of stoneblood',
 'num_auctions': 60,
 'quantity': 100,
 'weight_sell': 80.941456,
 'avg_sell': 37.06852666666666,
 'max': 293.993,
 'p80': 49.7998,
 'p50': 20.9997,
 'p20': 20.9994,
 'wp80': 104.9985,
 'wp50': 49.7998,
 'wp20': 20.9994,
 'min': 20.9993,
 'realm_market_value': 25.8601,
 'realm_historical': 14.9918,
 'region_historical': 10.9687,
 'region_avg_sale_price': 8.8059,
 'sale_pct': 51.3,
 'sold_per_day': 626,
 'headroom': 1120,
 'market_skew_pct': 72.49,
 'auction_skew_pct': -99.99}

In [36]:
approaches(item_info("flask of stoneblood"))

TypeError: expected string or bytes-like object

In [ ]:
item_info("icethorn")

In [ ]:
item_info("adder's tongue")